## Initialisation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from pathlib import Path
from random import Random
import cv2
import yaml
import numpy as np
from collections import Counter
from typing import List, Tuple

In [3]:
# --- config: change as needed ---
IMAGES_DIR = Path("/content/drive/MyDrive/dataset/images")
LABELS_DIR = Path("/content/drive/MyDrive/dataset/labels")
OUT_DIR = Path("dataset_split")
RNG = Random(42)

# class names (must match dataset order)
CLASS_NAMES = [
        "Canine (13)","Canine (23)","Canine (33)","Canine (43)",
        "Central Incisor (21)","Central Incisor (41)","Central Incisor (31)","Central Incisor (11)",
        "First Molar (16)","First Molar (26)","First Molar (36)","First Molar (46)",
        "First Premolar (14)","First Premolar (34)","First Premolar (44)","First Premolar (24)",
        "Lateral Incisor (22)","Lateral Incisor (32)","Lateral Incisor (42)","Lateral Incisor (12)",
        "Second Molar (17)","Second Molar (27)","Second Molar (37)","Second Molar (47)",
        "Second Premolar (15)","Second Premolar (25)","Second Premolar (35)","Second Premolar (45)",
        "Third Molar (18)","Third Molar (28)","Third Molar (38)","Third Molar (48)"
    ]

In [4]:
def list_image_files(img_dir: Path) -> List[Path]:
    return sorted([p for p in img_dir.iterdir()])

def read_labels_file(label_path: Path) -> List[Tuple[int, float, float, float, float]]:
    """Return list of (class, x_c, y_c, w, h) all floats; empty list if file missing/empty."""

    if not label_path.exists():
        return []
    out = []
    with open(label_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) != 5:
                # handle malformed line
                continue
            cls = int(parts[0])
            nums = list(map(float, parts[1:]))
            out.append((cls, *nums))
    return out

In [5]:
def yolo_to_bbox(yolo_box, img_w, img_h):
    """yolo_box = (x_c_norm, y_c_norm, w_norm, h_norm) -> (x_min,y_min,x_max,y_max) in pixels (int)"""
    xc, yc, w, h = yolo_box
    x1 = int((xc - w / 2.0) * img_w)
    y1 = int((yc - h / 2.0) * img_h)
    x2 = int((xc + w / 2.0) * img_w)
    y2 = int((yc + h / 2.0) * img_h)
    # clamp
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(img_w - 1, x2)
    y2 = min(img_h - 1, y2)
    return x1, y1, x2, y2

# def visualize_image_with_boxes(img_path: Path, labels: List[Tuple[int,float,float,float,float]], show=True, save_to=None):
#     img = cv2.imread(str(img_path))
#     h, w = img.shape[:2]
#     for cls, xc, yc, bw, bh in labels:
#         x1,y1,x2,y2 = yolo_to_bbox((xc,yc,bw,bh), w, h)
#         cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,0), 2)
#         txt = f"{cls}:{CLASS_NAMES[cls]}"
#         cv2.putText(img, txt, (x1, max(15,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, cv2.LINE_AA)
#     if save_to:
#         cv2.imwrite(str(save_to), img)
#     if show:
#         cv2.imshow("vis", img)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

In [6]:
def dataset_stats(img_dir=IMAGES_DIR, lbl_dir=LABELS_DIR):
    imgs = list_image_files(img_dir)
    missing_labels = []
    counts = Counter()
    per_image_counts = {}
    for img_p in imgs:
        lbl_p = lbl_dir / (img_p.stem + ".txt")
        labels = read_labels_file(lbl_p)
        per_image_counts[img_p.name] = len(labels)
        for cls, *_ in labels:
            counts[cls] += 1
        if not lbl_p.exists():
            missing_labels.append(img_p.name)
    return {
        "num_images": len(imgs),
        "per_class_counts": counts,
        "missing_label_files": missing_labels,
        "per_image_counts": per_image_counts
    }


In [7]:
def create_data_yaml(train_paths, val_paths, test_paths, out_path="data.yaml"):
    d = {
        "train": train_paths,
        "val": val_paths,
        "test": test_paths,
        "names": {i: CLASS_NAMES[i] for i in range(len(CLASS_NAMES))}
    }
    with open(out_path, "w") as f:
        yaml.safe_dump(d, f)
    print("Saved", out_path)

def split_dataset(img_dir=IMAGES_DIR, lbl_dir=LABELS_DIR, out_dir=OUT_DIR, seed=42, train_ratio=0.8, val_ratio=0.1):
    RNG.seed(seed)
    imgs = list_image_files(img_dir)
    RNG.shuffle(imgs)
    n = len(imgs)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)
    train = imgs[:n_train]
    val = imgs[n_train:n_train+n_val]
    test = imgs[n_train+n_val:]
    # make folders and copy files
    for subset, lst in [("train",train),("val",val),("test",test)]:
        im_out = out_dir / subset / "images"
        lb_out = out_dir / subset / "labels"
        im_out.mkdir(parents=True, exist_ok=True)
        lb_out.mkdir(parents=True, exist_ok=True)
        for p in lst:
            # copy image
            cv2.imwrite(str(im_out / p.name), cv2.imread(str(p)))
            # copy label (if missing, create empty)
            src_lbl = lbl_dir / (p.stem + ".txt")
            tgt_lbl = lb_out / (p.stem + ".txt")
            if src_lbl.exists():
                with open(src_lbl, "r") as fr, open(tgt_lbl, "w") as fw:
                    fw.write(fr.read())
            else:
                tgt_lbl.write_text("")
    print("Split complete: train/val/test sizes:", len(train), len(val), len(test))
    # create data.yaml
    create_data_yaml(str((out_dir/"train/images").resolve()),
                     str((out_dir/"val/images").resolve()),
                     str((out_dir/"test/images").resolve()),
                     out_path=str(out_dir/"data.yaml"))
    return {"train":train, "val":val, "test":test}

In [8]:
# stats = dataset_stats()
# print("Images:", stats["num_images"])
# print("Per-class counts:", stats["per_class_counts"])
# print("Missing label files:", len(stats["missing_label_files"]))

In [9]:
split_dataset()

Split complete: train/val/test sizes: 397 49 51
Saved dataset_split/data.yaml


{'train': [PosixPath('/content/drive/MyDrive/dataset/images/cate10-00072_jpg.rf.103bc7aee3af6b953a340774e274ee2b.jpg'),
  PosixPath('/content/drive/MyDrive/dataset/images/cate8-00123_jpg.rf.dc09677b0cc42d1f1d4f7b2619759349.jpg'),
  PosixPath('/content/drive/MyDrive/dataset/images/46f25eb3-20240716-144513555.jpg'),
  PosixPath('/content/drive/MyDrive/dataset/images/cate10-00056_jpg.rf.8122111fefd7c62440c200a8d0d004c2.jpg'),
  PosixPath('/content/drive/MyDrive/dataset/images/54aabd9e-20240530-153639805.jpg'),
  PosixPath('/content/drive/MyDrive/dataset/images/73ffa9a1-20240615-122751488.jpg'),
  PosixPath('/content/drive/MyDrive/dataset/images/cate8-00397_jpg.rf.8ccc23412af94ef90976e09ce1f4982e.jpg'),
  PosixPath('/content/drive/MyDrive/dataset/images/cate2-00027_jpg.rf.cf608bef7b2d8bb65a96dbc4d793ccd7.jpg'),
  PosixPath('/content/drive/MyDrive/dataset/images/74d3673e-20250502-143252934.jpg'),
  PosixPath('/content/drive/MyDrive/dataset/images/cate7-00025_jpg.rf.00d895357624d877d0ad3b23e

In [10]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00


## Yolov8

In [11]:
from ultralytics import YOLO
model = YOLO('yolov8s.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model.train(data='/content/dataset_split/data.yaml', epochs=50, imgsz=640, batch=16, project='experiments', name='baseline')

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_split/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=baseline, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pre

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79ffa6ec8e90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,

In [ ]:
ckpt = list(Path("experiments").glob("**/weights/best.pt"))[-1]
print("Using checkpoint:", ckpt)
m = YOLO(str(ckpt))
val_results = m.val(data="/content/dataset_split/data.yaml", imgsz=640)
# ultralytics prints per-class AP; val_results contains returned objects
print("Validation finished. See printed per-class AP above.")

Using checkpoint: experiments/baseline/weights/best.pt
Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,137,968 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2081.6±847.0 MB/s, size: 111.4 KB)
val: Scanning /content/dataset_split/val/labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 49/49 82871.3it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.87it/s 4.6s
                   all         49       1353      0.874      0.882      0.922      0.647
           Canine (13)         44         44      0.865      0.886      0.904      0.645
           Canine (23)         44         44      0.838      0.939      0.911      0.636
           Canine (33)         48         48      0.896      0.895      0.924      0.625
           Canine (43)         48         48      0.877      0.917      0.896 

In [17]:
import json
from collections import defaultdict, Counter

In [18]:
OUT = Path("outputs")
OUT.mkdir(exist_ok=True)

In [ ]:
# set this to your checkpoint (best.pt) if not auto-found
ckpt_candidates = list(Path("experiments").glob("**/weights/best.pt"))
if len(ckpt_candidates) == 0:
    ckpt = "best.pt"   # change to your path if needed
else:
    ckpt = str(ckpt_candidates[-1])
print("Using checkpoint:", ckpt)

model = YOLO(ckpt)
results = model.predict(source=str("/content/dataset_split/test/images"), conf=0.25, imgsz=640, save=False)  # runs inference on test folder

preds = []
for r in results:
    img_path = Path(r.path)
    img_h, img_w = int(r.orig_shape[0]), int(r.orig_shape[1])
    for b in r.boxes:
        cls = int(b.cls.cpu().numpy()[0])
        conf = float(b.conf.cpu().numpy()[0])
        x1,y1,x2,y2 = b.xyxy.cpu().numpy()[0].tolist()
        preds.append({
            "image": str(img_path),
            "class": cls,
            "conf": conf,
            "x1": float(x1), "y1": float(y1), "x2": float(x2), "y2": float(y2)
        })

with open(OUT/"preds.json", "w") as f:
    json.dump(preds, f, indent=2)
print("Saved preds:", OUT/"preds.json", "Total preds:", len(preds))

Using checkpoint: experiments/baseline/weights/best.pt

image 1/51 /content/dataset_split/test/images/05ff01fa-20250108-115407401.jpg: 640x640 2 Canine (13)s, 1 Canine (33), 1 Canine (43), 2 Central Incisor (41)s, 1 Central Incisor (31), 2 Central Incisor (11)s, 2 First Molar (16)s, 1 First Molar (36), 1 First Molar (46), 2 First Premolar (14)s, 1 First Premolar (34), 1 First Premolar (44), 1 Lateral Incisor (42), 2 Lateral Incisor (12)s, 3 Second Molar (17)s, 1 Second Molar (37), 2 Second Molar (47)s, 2 Second Premolar (15)s, 1 Second Premolar (35), 1 Second Premolar (45), 2 Third Molar (18)s, 1 Third Molar (38), 1 Third Molar (48), 16.0ms
image 2/51 /content/dataset_split/test/images/124e696d-20240914-105651782.jpg: 640x640 2 Canine (13)s, 1 Canine (33), 2 Central Incisor (41)s, 1 Central Incisor (31), 1 Central Incisor (11), 2 First Molar (16)s, 1 First Molar (36), 1 First Molar (46), 2 First Premolar (14)s, 1 First Premolar (34), 1 First Premolar (44), 1 Lateral Incisor (32), 1 Lat

In [ ]:
def read_yolo(lbl_path, img_w, img_h):
    lines = lbl_path.read_text().strip().splitlines() if lbl_path.exists() else []
    out = []
    for L in lines:
        parts = L.split()
        cls = int(parts[0])
        xc, yc, w, h = map(float, parts[1:])
        x1 = (xc - w/2)*img_w
        y1 = (yc - h/2)*img_h
        x2 = (xc + w/2)*img_w
        y2 = (yc + h/2)*img_h
        out.append({"class": cls, "x1": x1, "y1": y1, "x2": x2, "y2": y2})
    return out

def iou_xyxy(a, b):
    x_left = max(a["x1"], b["x1"])
    y_top = max(a["y1"], b["y1"])
    x_right = min(a["x2"], b["x2"])
    y_bottom = min(a["y2"], b["y2"])
    if x_right <= x_left or y_bottom <= y_top:
        return 0.0
    inter = (x_right - x_left) * (y_bottom - y_top)
    area_a = (a["x2"]-a["x1"])*(a["y2"]-a["y1"])
    area_b = (b["x2"]-b["x1"])*(b["y2"]-b["y1"])
    return inter / (area_a + area_b - inter + 1e-9)

# Build GT dictionary: image_path -> list of GT boxes
TEST_ROOT = Path("/content/dataset_split/test")
TEST_IMGS = TEST_ROOT / "images"
TEST_LABELS = TEST_ROOT / "labels"
gt_by_image = {}
for img_p in sorted(TEST_IMGS.iterdir()):
    img = cv2.imread(str(img_p))
    h,w = img.shape[:2]
    lbl = TEST_LABELS / (img_p.stem + ".txt")
    gt_by_image[str(img_p)] = read_yolo(lbl, w, h)

# quick check
total_gt = sum(len(v) for v in gt_by_image.values())
print("Total ground-truth boxes:", total_gt)

Total ground-truth boxes: 1485


In [ ]:
with open(OUT/"preds.json") as f:
    preds = json.load(f)

# group predictions by class and sort descending confidence
preds_by_class = defaultdict(list)
for p in preds:
    preds_by_class[p["class"]].append(p)
for cls in preds_by_class:
    preds_by_class[cls] = sorted(preds_by_class[cls], key=lambda x: -x["conf"])

# count GT per class and keep per-image matched flags
gt_count_per_class = Counter()
gt_matched = {}  # (img_path, gt_idx) -> matched bool
for img_path, gts in gt_by_image.items():
    for i, g in enumerate(gts):
        gt_count_per_class[g["class"]] += 1
        gt_matched[(img_path, i)] = False

IOU_T = 0.5
per_class_metrics = {}
for cls, pred_list in preds_by_class.items():
    tp = []
    fp = []
    matched_this_class = 0
    for p in pred_list:
        img = p["image"]
        # find GT boxes in same image of same class that are not matched
        gts = [ (i,g) for i,g in enumerate(gt_by_image.get(img,[])) if g["class"]==cls and not gt_matched[(img,i)] ]
        best_iou = 0.0
        best_idx = None
        for i,g in gts:
            iou = iou_xyxy(p, g)
            if iou > best_iou:
                best_iou = iou
                best_idx = i
        if best_iou >= IOU_T:
            tp.append(1)
            fp.append(0)
            gt_matched[(img,best_idx)] = True
            matched_this_class += 1
        else:
            tp.append(0)
            fp.append(1)
    tp_cum = np.cumsum(tp)
    fp_cum = np.cumsum(fp)
    if len(tp_cum) == 0:
        precisions = np.array([])
        recalls = np.array([])
        ap = 0.0
    else:
        precisions = tp_cum / (tp_cum + fp_cum + 1e-9)
        recalls = tp_cum / (gt_count_per_class[cls] + 1e-9)
        # ensure monotonic recall axis for trapz: prepend (0,1) point if needed
        ap = np.trapezoid(precisions, recalls) if recalls.size>1 else (precisions[0] if precisions.size==1 else 0.0)
    per_class_metrics[cls] = {
        "GT": int(gt_count_per_class[cls]),
        "Preds": len(pred_list),
        "TP": int(matched_this_class),
        "FP": int(len(pred_list) - matched_this_class),
        "Precision_at_each_pred_sample": precisions.tolist(),
        "Recall_at_each_pred_sample": recalls.tolist(),
        "AP_approx": float(ap)
    }

# compute mean AP across classes with at least one GT
ap_list = [v["AP_approx"] for k,v in per_class_metrics.items() if v["GT"]>0]
mAP = float(np.mean(ap_list)) if ap_list else 0.0

# print summary
for cls in sorted(per_class_metrics.keys()):
    m = per_class_metrics[cls]
    print(f"Class {cls}: GT={m['GT']}, Preds={m['Preds']}, TP={m['TP']}, FP={m['FP']}, AP~={m['AP_approx']:.3f}")

print(f"\nApproximate mAP (IoU=0.5) over classes with GT: {mAP:.4f}")

Class 0: GT=50, Preds=63, TP=47, FP=16, AP~=0.834
Class 1: GT=49, Preds=49, TP=40, FP=9, AP~=0.713
Class 2: GT=49, Preds=51, TP=38, FP=13, AP~=0.711
Class 3: GT=50, Preds=57, TP=47, FP=10, AP~=0.839
Class 4: GT=48, Preds=44, TP=40, FP=4, AP~=0.775
Class 5: GT=51, Preds=69, TP=47, FP=22, AP~=0.782
Class 6: GT=49, Preds=61, TP=42, FP=19, AP~=0.772
Class 7: GT=49, Preds=57, TP=47, FP=10, AP~=0.839
Class 8: GT=47, Preds=57, TP=45, FP=12, AP~=0.841
Class 9: GT=44, Preds=40, TP=37, FP=3, AP~=0.789
Class 10: GT=45, Preds=41, TP=37, FP=4, AP~=0.795
Class 11: GT=45, Preds=48, TP=41, FP=7, AP~=0.829
Class 12: GT=48, Preds=58, TP=43, FP=15, AP~=0.769
Class 13: GT=47, Preds=48, TP=39, FP=9, AP~=0.725
Class 14: GT=48, Preds=56, TP=45, FP=11, AP~=0.856
Class 15: GT=48, Preds=47, TP=39, FP=8, AP~=0.739
Class 16: GT=48, Preds=44, TP=39, FP=5, AP~=0.690
Class 17: GT=49, Preds=50, TP=38, FP=12, AP~=0.712
Class 18: GT=51, Preds=56, TP=44, FP=12, AP~=0.793
Class 19: GT=48, Preds=61, TP=47, FP=14, AP~=0.91

## Yolov9n

In [22]:
model = YOLO('yolov9c.pt')

In [24]:
model.train(data='/content/dataset_split/data.yaml', epochs=100, imgsz=640, batch=16, project='experiments', name='yolov9c_baseline')

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_split/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9c.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov9c_baseline2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c32d0fada30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,

In [25]:
# 1. Explicitly define the path to your best model's checkpoint
ckpt = "experiments/yolov9c_baseline2/weights/best.pt"
print("Using checkpoint:", ckpt)

# 2. Load the model from the specified checkpoint
m = YOLO(str(ckpt))

# 3. Run evaluation on the TEST set for a final, unbiased score
print("\\nEvaluating on the TEST set...")
test_results = m.val(data="/content/dataset_split/data.yaml", imgsz=640, split='test')

print("\\nTest finished. The printed per-class AP above is the final performance on unseen data.")

Using checkpoint: experiments/yolov9c_baseline2/weights/best.pt
\nEvaluating on the TEST set...
Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9c summary (fused): 156 layers, 25,343,920 parameters, 0 gradients, 102.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1699.4±210.1 MB/s, size: 118.6 KB)
val: Scanning /content/dataset_split/test/labels.cache... 51 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 51/51 87956.2it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.65it/s 6.1s
                   all         51       1485      0.926      0.937       0.96      0.696
           Canine (13)         50         50      0.958       0.96      0.977      0.706
           Canine (23)         48         49       0.92      0.933       0.96      0.665
           Canine (33)         49         49      0.866      0.878      0.884      0.646
           Canine (43)         5

In [26]:
# set this to your checkpoint (best.pt) if not auto-found
ckpt_candidates = list(Path("experiments/yolov9c_baseline2").glob("**/weights/best.pt"))
if len(ckpt_candidates) == 0:
    ckpt = "best.pt"   # change to your path if needed
else:
    ckpt = str(ckpt_candidates[-1])
print("Using checkpoint:", ckpt)

model = YOLO(ckpt)
results = model.predict(source=str("/content/dataset_split/test/images"), conf=0.25, imgsz=640, save=False)  # runs inference on test folder

preds = []
for r in results:
    img_path = Path(r.path)
    img_h, img_w = int(r.orig_shape[0]), int(r.orig_shape[1])
    for b in r.boxes:
        cls = int(b.cls.cpu().numpy()[0])
        conf = float(b.conf.cpu().numpy()[0])
        x1,y1,x2,y2 = b.xyxy.cpu().numpy()[0].tolist()
        preds.append({
            "image": str(img_path),
            "class": cls,
            "conf": conf,
            "x1": float(x1), "y1": float(y1), "x2": float(x2), "y2": float(y2)
        })

with open(OUT/"preds_yolov9c.json", "w") as f:
    json.dump(preds, f, indent=2)
print("Saved preds:", OUT/"preds_yolov9c.json", "Total preds:", len(preds))

Using checkpoint: experiments/yolov9c_baseline2/weights/best.pt

image 1/51 /content/dataset_split/test/images/05ff01fa-20250108-115407401.jpg: 640x640 1 Canine (13), 1 Canine (23), 1 Canine (33), 2 Canine (43)s, 1 Central Incisor (21), 1 Central Incisor (41), 1 Central Incisor (31), 1 Central Incisor (11), 1 First Molar (16), 1 First Molar (26), 1 First Molar (36), 2 First Molar (46)s, 1 First Premolar (14), 1 First Premolar (34), 2 First Premolar (44)s, 1 First Premolar (24), 1 Lateral Incisor (22), 1 Lateral Incisor (32), 1 Lateral Incisor (42), 1 Lateral Incisor (12), 1 Second Molar (17), 1 Second Molar (27), 1 Second Molar (37), 1 Second Premolar (15), 1 Second Premolar (25), 1 Second Premolar (35), 2 Second Premolar (45)s, 1 Third Molar (18), 1 Third Molar (28), 1 Third Molar (38), 1 Third Molar (48), 49.1ms
image 2/51 /content/dataset_split/test/images/124e696d-20240914-105651782.jpg: 640x640 1 Canine (13), 1 Canine (23), 1 Canine (33), 1 Canine (43), 1 Central Incisor (21), 1 C

In [27]:
def read_yolo(lbl_path, img_w, img_h):
    lines = lbl_path.read_text().strip().splitlines() if lbl_path.exists() else []
    out = []
    for L in lines:
        parts = L.split()
        cls = int(parts[0])
        xc, yc, w, h = map(float, parts[1:])
        x1 = (xc - w/2)*img_w
        y1 = (yc - h/2)*img_h
        x2 = (xc + w/2)*img_w
        y2 = (yc + h/2)*img_h
        out.append({"class": cls, "x1": x1, "y1": y1, "x2": x2, "y2": y2})
    return out

def iou_xyxy(a, b):
    x_left = max(a["x1"], b["x1"])
    y_top = max(a["y1"], b["y1"])
    x_right = min(a["x2"], b["x2"])
    y_bottom = min(a["y2"], b["y2"])
    if x_right <= x_left or y_bottom <= y_top:
        return 0.0
    inter = (x_right - x_left) * (y_bottom - y_top)
    area_a = (a["x2"]-a["x1"])*(a["y2"]-a["y1"])
    area_b = (b["x2"]-b["x1"])*(b["y2"]-b["y1"])
    return inter / (area_a + area_b - inter + 1e-9)

# Build GT dictionary: image_path -> list of GT boxes
TEST_ROOT = Path("/content/dataset_split/test")
TEST_IMGS = TEST_ROOT / "images"
TEST_LABELS = TEST_ROOT / "labels"
gt_by_image = {}
for img_p in sorted(TEST_IMGS.iterdir()):
    img = cv2.imread(str(img_p))
    h,w = img.shape[:2]
    lbl = TEST_LABELS / (img_p.stem + ".txt")
    gt_by_image[str(img_p)] = read_yolo(lbl, w, h)

# quick check
total_gt = sum(len(v) for v in gt_by_image.values())
print("Total ground-truth boxes:", total_gt)

Total ground-truth boxes: 1485


In [28]:
with open(OUT/"preds_yolov9c.json") as f:
    preds = json.load(f)

# group predictions by class and sort descending confidence
preds_by_class = defaultdict(list)
for p in preds:
    preds_by_class[p["class"]].append(p)
for cls in preds_by_class:
    preds_by_class[cls] = sorted(preds_by_class[cls], key=lambda x: -x["conf"])

# count GT per class and keep per-image matched flags
gt_count_per_class = Counter()
gt_matched = {}  # (img_path, gt_idx) -> matched bool
for img_path, gts in gt_by_image.items():
    for i, g in enumerate(gts):
        gt_count_per_class[g["class"]] += 1
        gt_matched[(img_path, i)] = False

IOU_T = 0.5
per_class_metrics = {}
for cls, pred_list in preds_by_class.items():
    tp = []
    fp = []
    matched_this_class = 0
    for p in pred_list:
        img = p["image"]
        # find GT boxes in same image of same class that are not matched
        gts = [ (i,g) for i,g in enumerate(gt_by_image.get(img,[])) if g["class"]==cls and not gt_matched[(img,i)] ]
        best_iou = 0.0
        best_idx = None
        for i,g in gts:
            iou = iou_xyxy(p, g)
            if iou > best_iou:
                best_iou = iou
                best_idx = i
        if best_iou >= IOU_T:
            tp.append(1)
            fp.append(0)
            gt_matched[(img,best_idx)] = True
            matched_this_class += 1
        else:
            tp.append(0)
            fp.append(1)
    tp_cum = np.cumsum(tp)
    fp_cum = np.cumsum(fp)
    if len(tp_cum) == 0:
        precisions = np.array([])
        recalls = np.array([])
        ap = 0.0
    else:
        precisions = tp_cum / (tp_cum + fp_cum + 1e-9)
        recalls = tp_cum / (gt_count_per_class[cls] + 1e-9)
        # ensure monotonic recall axis for trapz: prepend (0,1) point if needed
        ap = np.trapezoid(precisions, recalls) if recalls.size>1 else (precisions[0] if precisions.size==1 else 0.0)
    per_class_metrics[cls] = {
        "GT": int(gt_count_per_class[cls]),
        "Preds": len(pred_list),
        "TP": int(matched_this_class),
        "FP": int(len(pred_list) - matched_this_class),
        "Precision_at_each_pred_sample": precisions.tolist(),
        "Recall_at_each_pred_sample": recalls.tolist(),
        "AP_approx": float(ap)
    }

# compute mean AP across classes with at least one GT
ap_list = [v["AP_approx"] for k,v in per_class_metrics.items() if v["GT"]>0]
mAP = float(np.mean(ap_list)) if ap_list else 0.0

# print summary
for cls in sorted(per_class_metrics.keys()):
    m = per_class_metrics[cls]
    print(f"Class {cls}: GT={m['GT']}, Preds={m['Preds']}, TP={m['TP']}, FP={m['FP']}, AP~={m['AP_approx']:.3f}")

print(f"\nApproximate mAP (IoU=0.5) over classes with GT: {mAP:.4f}")

Class 0: GT=50, Preds=52, TP=49, FP=3, AP~=0.938
Class 1: GT=49, Preds=49, TP=46, FP=3, AP~=0.900
Class 2: GT=49, Preds=52, TP=44, FP=8, AP~=0.848
Class 3: GT=50, Preds=54, TP=47, FP=7, AP~=0.903
Class 4: GT=48, Preds=48, TP=45, FP=3, AP~=0.890
Class 5: GT=51, Preds=51, TP=48, FP=3, AP~=0.908
Class 6: GT=49, Preds=50, TP=46, FP=4, AP~=0.903
Class 7: GT=49, Preds=51, TP=48, FP=3, AP~=0.951
Class 8: GT=47, Preds=50, TP=47, FP=3, AP~=0.958
Class 9: GT=44, Preds=43, TP=41, FP=2, AP~=0.909
Class 10: GT=45, Preds=46, TP=43, FP=3, AP~=0.930
Class 11: GT=45, Preds=48, TP=43, FP=5, AP~=0.905
Class 12: GT=48, Preds=52, TP=46, FP=6, AP~=0.930
Class 13: GT=47, Preds=49, TP=44, FP=5, AP~=0.904
Class 14: GT=48, Preds=52, TP=47, FP=5, AP~=0.958
Class 15: GT=48, Preds=48, TP=45, FP=3, AP~=0.914
Class 16: GT=48, Preds=48, TP=45, FP=3, AP~=0.879
Class 17: GT=49, Preds=47, TP=43, FP=4, AP~=0.834
Class 18: GT=51, Preds=52, TP=48, FP=4, AP~=0.911
Class 19: GT=48, Preds=48, TP=47, FP=1, AP~=0.956
Class 20: 

## Yolov9 with hyp.scratch-high.yaml

In [33]:
model = YOLO('yolov9c.pt')

In [35]:
model.train(
    data='/content/dataset_split/data.yaml',
    epochs=150,
    imgsz=640,
    batch=16,
    project='experiments',
    name='yolov9n_high_aug_corrected',

    # --- High-Augmentation Hyperparameters ---
    degrees=10.0,      # image rotation (+/- deg)
    translate=0.2,     # image translation (+/- fraction)
    scale=0.9,         # image scale (+/- gain)
    shear=10.0,        # image shear (+/- deg)
    perspective=0.001, # image perspective (+/- fraction), range 0-0.001
    flipud=0.1,        # image flip up-down (probability)
    fliplr=0.5,        # image flip left-right (probability)
    mosaic=1.0,        # mosaic augmentation (probability)
    mixup=0.15,        # mixup augmentation (probability)
    copy_paste=0.3     # copy-paste augmentation (probability)
)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_split/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.15, mode=train, model=yolov9c.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov9n_high_aug_corrected, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.001, plo

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c345c19fd10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,

In [36]:
# 1. Explicitly define the path to your best model's checkpoint
ckpt = "experiments/yolov9n_high_aug_corrected/weights/best.pt"
print("Using checkpoint:", ckpt)

# 2. Load the model from the specified checkpoint
m = YOLO(str(ckpt))

# 3. Run evaluation on the TEST set for a final, unbiased score
print("\\nEvaluating on the TEST set...")
test_results = m.val(data="/content/dataset_split/data.yaml", imgsz=640, split='test')

print("\\nTest finished. The printed per-class AP above is the final performance on unseen data.")

Using checkpoint: experiments/yolov9n_high_aug_corrected/weights/best.pt
\nEvaluating on the TEST set...
Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9c summary (fused): 156 layers, 25,343,920 parameters, 0 gradients, 102.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1639.9±609.8 MB/s, size: 118.6 KB)
val: Scanning /content/dataset_split/test/labels.cache... 51 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 51/51 92004.1it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.73it/s 5.5s
                   all         51       1485      0.868      0.904      0.942      0.635
           Canine (13)         50         50      0.884       0.94       0.96      0.644
           Canine (23)         48         49      0.882      0.915       0.93      0.579
           Canine (33)         49         49      0.836      0.857      0.865      0.568
           Canine (43) 

In [40]:
ckpt = "experiments/yolov9n_high_aug_corrected/weights/best.pt"
model = YOLO(ckpt)
results = model.predict(source=str("/content/dataset_split/test/images"), conf=0.25, imgsz=640, save=False)  # runs inference on test folder

preds = []
for r in results:
    img_path = Path(r.path)
    img_h, img_w = int(r.orig_shape[0]), int(r.orig_shape[1])
    for b in r.boxes:
        cls = int(b.cls.cpu().numpy()[0])
        conf = float(b.conf.cpu().numpy()[0])
        x1,y1,x2,y2 = b.xyxy.cpu().numpy()[0].tolist()
        preds.append({
            "image": str(img_path),
            "class": cls,
            "conf": conf,
            "x1": float(x1), "y1": float(y1), "x2": float(x2), "y2": float(y2)
        })

with open(OUT/"preds_yolov9_high_aug.json", "w") as f:
    json.dump(preds, f, indent=2)
print("Saved preds:", OUT/"preds_yolov9_high_aug.json", "Total preds:", len(preds))


image 1/51 /content/dataset_split/test/images/05ff01fa-20250108-115407401.jpg: 640x640 1 Canine (13), 1 Canine (23), 1 Canine (33), 1 Canine (43), 1 Central Incisor (21), 1 Central Incisor (41), 1 Central Incisor (31), 1 Central Incisor (11), 1 First Molar (16), 1 First Molar (26), 1 First Molar (36), 1 First Molar (46), 1 First Premolar (14), 1 First Premolar (34), 1 First Premolar (44), 1 First Premolar (24), 1 Lateral Incisor (22), 1 Lateral Incisor (32), 1 Lateral Incisor (42), 1 Lateral Incisor (12), 1 Second Molar (17), 2 Second Molar (27)s, 1 Second Molar (37), 1 Second Molar (47), 1 Second Premolar (15), 1 Second Premolar (25), 1 Second Premolar (35), 1 Second Premolar (45), 2 Third Molar (28)s, 2 Third Molar (48)s, 47.9ms
image 2/51 /content/dataset_split/test/images/124e696d-20240914-105651782.jpg: 640x640 1 Canine (13), 2 Canine (23)s, 1 Canine (33), 1 Canine (43), 1 Central Incisor (21), 1 Central Incisor (41), 1 Central Incisor (31), 1 Central Incisor (11), 1 First Molar 

In [41]:
def read_yolo(lbl_path, img_w, img_h):
    lines = lbl_path.read_text().strip().splitlines() if lbl_path.exists() else []
    out = []
    for L in lines:
        parts = L.split()
        cls = int(parts[0])
        xc, yc, w, h = map(float, parts[1:])
        x1 = (xc - w/2)*img_w
        y1 = (yc - h/2)*img_h
        x2 = (xc + w/2)*img_w
        y2 = (yc + h/2)*img_h
        out.append({"class": cls, "x1": x1, "y1": y1, "x2": x2, "y2": y2})
    return out

def iou_xyxy(a, b):
    x_left = max(a["x1"], b["x1"])
    y_top = max(a["y1"], b["y1"])
    x_right = min(a["x2"], b["x2"])
    y_bottom = min(a["y2"], b["y2"])
    if x_right <= x_left or y_bottom <= y_top:
        return 0.0
    inter = (x_right - x_left) * (y_bottom - y_top)
    area_a = (a["x2"]-a["x1"])*(a["y2"]-a["y1"])
    area_b = (b["x2"]-b["x1"])*(b["y2"]-b["y1"])
    return inter / (area_a + area_b - inter + 1e-9)

# Build GT dictionary: image_path -> list of GT boxes
TEST_ROOT = Path("/content/dataset_split/test")
TEST_IMGS = TEST_ROOT / "images"
TEST_LABELS = TEST_ROOT / "labels"
gt_by_image = {}
for img_p in sorted(TEST_IMGS.iterdir()):
    img = cv2.imread(str(img_p))
    h,w = img.shape[:2]
    lbl = TEST_LABELS / (img_p.stem + ".txt")
    gt_by_image[str(img_p)] = read_yolo(lbl, w, h)

# quick check
total_gt = sum(len(v) for v in gt_by_image.values())
print("Total ground-truth boxes:", total_gt)

Total ground-truth boxes: 1485


In [42]:
with open(OUT/"preds_yolov9_high_aug.json") as f:
    preds = json.load(f)

# group predictions by class and sort descending confidence
preds_by_class = defaultdict(list)
for p in preds:
    preds_by_class[p["class"]].append(p)
for cls in preds_by_class:
    preds_by_class[cls] = sorted(preds_by_class[cls], key=lambda x: -x["conf"])

# count GT per class and keep per-image matched flags
gt_count_per_class = Counter()
gt_matched = {}  # (img_path, gt_idx) -> matched bool
for img_path, gts in gt_by_image.items():
    for i, g in enumerate(gts):
        gt_count_per_class[g["class"]] += 1
        gt_matched[(img_path, i)] = False

IOU_T = 0.5
per_class_metrics = {}
for cls, pred_list in preds_by_class.items():
    tp = []
    fp = []
    matched_this_class = 0
    for p in pred_list:
        img = p["image"]
        # find GT boxes in same image of same class that are not matched
        gts = [ (i,g) for i,g in enumerate(gt_by_image.get(img,[])) if g["class"]==cls and not gt_matched[(img,i)] ]
        best_iou = 0.0
        best_idx = None
        for i,g in gts:
            iou = iou_xyxy(p, g)
            if iou > best_iou:
                best_iou = iou
                best_idx = i
        if best_iou >= IOU_T:
            tp.append(1)
            fp.append(0)
            gt_matched[(img,best_idx)] = True
            matched_this_class += 1
        else:
            tp.append(0)
            fp.append(1)
    tp_cum = np.cumsum(tp)
    fp_cum = np.cumsum(fp)
    if len(tp_cum) == 0:
        precisions = np.array([])
        recalls = np.array([])
        ap = 0.0
    else:
        precisions = tp_cum / (tp_cum + fp_cum + 1e-9)
        recalls = tp_cum / (gt_count_per_class[cls] + 1e-9)
        # ensure monotonic recall axis for trapz: prepend (0,1) point if needed
        ap = np.trapezoid(precisions, recalls) if recalls.size>1 else (precisions[0] if precisions.size==1 else 0.0)
    per_class_metrics[cls] = {
        "GT": int(gt_count_per_class[cls]),
        "Preds": len(pred_list),
        "TP": int(matched_this_class),
        "FP": int(len(pred_list) - matched_this_class),
        "Precision_at_each_pred_sample": precisions.tolist(),
        "Recall_at_each_pred_sample": recalls.tolist(),
        "AP_approx": float(ap)
    }

# compute mean AP across classes with at least one GT
ap_list = [v["AP_approx"] for k,v in per_class_metrics.items() if v["GT"]>0]
mAP = float(np.mean(ap_list)) if ap_list else 0.0

# print summary
for cls in sorted(per_class_metrics.keys()):
    m = per_class_metrics[cls]
    print(f"Class {cls}: GT={m['GT']}, Preds={m['Preds']}, TP={m['TP']}, FP={m['FP']}, AP~={m['AP_approx']:.3f}")

print(f"\nApproximate mAP (IoU=0.5) over classes with GT: {mAP:.4f}")

Class 0: GT=50, Preds=55, TP=47, FP=8, AP~=0.890
Class 1: GT=49, Preds=52, TP=45, FP=7, AP~=0.882
Class 2: GT=49, Preds=52, TP=42, FP=10, AP~=0.770
Class 3: GT=50, Preds=51, TP=45, FP=6, AP~=0.834
Class 4: GT=48, Preds=50, TP=44, FP=6, AP~=0.887
Class 5: GT=51, Preds=54, TP=48, FP=6, AP~=0.913
Class 6: GT=49, Preds=50, TP=45, FP=5, AP~=0.874
Class 7: GT=49, Preds=54, TP=46, FP=8, AP~=0.895
Class 8: GT=47, Preds=51, TP=46, FP=5, AP~=0.947
Class 9: GT=44, Preds=48, TP=41, FP=7, AP~=0.890
Class 10: GT=45, Preds=50, TP=42, FP=8, AP~=0.889
Class 11: GT=45, Preds=47, TP=41, FP=6, AP~=0.872
Class 12: GT=48, Preds=55, TP=45, FP=10, AP~=0.904
Class 13: GT=47, Preds=50, TP=42, FP=8, AP~=0.840
Class 14: GT=48, Preds=51, TP=46, FP=5, AP~=0.915
Class 15: GT=48, Preds=52, TP=42, FP=10, AP~=0.834
Class 16: GT=48, Preds=49, TP=42, FP=7, AP~=0.830
Class 17: GT=49, Preds=51, TP=42, FP=9, AP~=0.794
Class 18: GT=51, Preds=54, TP=46, FP=8, AP~=0.872
Class 19: GT=48, Preds=53, TP=45, FP=8, AP~=0.905
Class 2